# 创建 DataFrame

## 使用 Python dict 建立 DataFrame

略。

## 使用 pd.util.testing 建立随机 DataFrame

In [ ]:
# import pandas as pd

# pd.util.testing.makeDataFrame().head(10)

## 把剪切板的内容转换成 DataFrame

其实质是复制表格内容，然后将其转换成 DataFrame.

Windows 下推荐视同复制工具：[Ditto clipboard manager](https://ditto-cp.sourceforge.io/).

步骤：

- 复制表格内容

- 执行 pd.read_clipboard()

演练：

复制上面随机生成的 DataFrame

In [ ]:
df = pd.read_clipboard()
df
# 自动把第一行当做 columns

df.to_csv("./output/randomDataFrame.csv")

## 读取在线 CSV 文档

略。

优点：方便。

缺点：URL 容易更改，从而导致失效。

备注：Chartify 的数据可视化效果似乎不错，可以考虑学习一下。

In [ ]:
# df = pd.read_csv('http://bit.ly/kaggletrain')
# df.head()

## 优化内存

### 查看已使用内存

In [ ]:
df.info(memory_usage="depp")

或者使用 Jupyter Notebook 的插件 Variable Inspector。

视频操作如下：
https://leemeng.tw/images/pandas/variable_inspector.mp4

### 优化内存

如果数据量较小，可使用 `category`转换已知类型的 column。

In [ ]:
dtypes = {"Embarked": "category"}
cols = ['PassengerId', 'Name', 'Sex', 'Embarked']
df = pd.read_csv('http://bit.ly/kaggletrain', 
                 dtype=dtypes, usecols=cols)
df.info(memory_usage="deep")

如果数据量大，而且所使用的内存小，则可以使用 `chunksize` 参数来限制每次读入的 rows.

In [ ]:
import pandas as pd
import numpy as np

from IPython.display import display

# chunksize=4 表示一次讀入 4 rows
df = pd.read_csv('titanic_train.csv', 
                     chunksize=4)
# 显示前兩個 chunks
for _, df_partial in zip(range(2), df):
    display(df_partial)

## 读入并合并多个 CSV 档案成单一 DataFrame

使用 `pd.concat()` 方法实现此需求。

并且，可以使用`pd.reset_index`重置新 DataFrame 的索引。

In [ ]:
from glob import glob
files = glob("dataset/passenger*.csv")

df = pd.concat([pd.read_csv(f) for f in files])
df.reset_index(drop=True)

# 自定义 DataFrame 显示设定

## 显示所有 Column

处理前：

In [ ]:
df = pd.util.testing.makeCustomDataframe(5, 25)
df

处理后：

①更改`display.max_columns`设定

In [ ]:
pd.set_option("display.max_columns", None)
df

②使用`.T`转置数据

In [ ]:
df.T

### 设置 Column 的宽度

In [ ]:
print("display.max_colwidth 的默认值是：", pd.get_option("display.max_colwidth"))
print("更改前的数据展示如下：")
display(df.head())

print("更改后的数据展示如下：")
pd.set_option("display.max_colwidth", 5)
display(df.head())

## 改变浮点数显示位数

In [ ]:
df = pd.read_csv("titanic_train.csv")

In [ ]:
pd.set_option("display.max_colwidth", 50)
pd.set_option("display.precision", 1)
df.head(3)

# 初始化所有设定
# pd.reset_option("all")

## 为特定 DataFrame 添加样式

In [ ]:
df_sample = df.sample(n=10, random_state=9527).drop('Name', axis=1)
df_sample.Age.fillna(int(df.Age.mean()), inplace=True)
df_sample

使用`.style`实现为特点 DataFrame 添加样式

In [ ]:
(df_sample.style
          .format('{:.1f}', subset='Fare')
          .set_caption("★五颜六色の泰坦尼克号数据集☆")
          .hide_index()
          .bar('Age', vmin=0)
          .highlight_max('Survived')
          .background_gradient('Greens',
                               subset='Fare')
          .highlight_null())

# 数据清理 & 整理

## 处理空值

略。

## 舍弃不需要的行列

略。

## 重置并舍弃索引

略。

## 将字符串切割成多个 Column

In [ ]:
df = pd.DataFrame({
    "name": ["大雄", "胖虎"], 
    "feature": ["膽小, 翻花繩", "粗魯, 演唱會"]
})
df

下面将演示如何将`feature`分成不同的 column。

In [ ]:
df[['性格', '特技']] = df.feature.str.split(',', expand=True)
df

步骤细解：
- 利用`str`取出字符串
- 使用`split`切割字符串
- 设置`expand=True`从而将结果扩大成 DataFrame

## 将 list 分成多个 column

In [ ]:
df = pd.DataFrame({
    'name': ["大雄", "胖虎"],
    'feature': [["胆小", "翻花绳"], ["粗鲁", "演唱会"]]
})
df

①使用`tolist`函数

In [ ]:
cols = ['性格', '特技']
pd.DataFrame(df.feature.tolist(), columns=cols)

②使用`apply(pd.Series)

In [ ]:
df.feature.apply(pd.Series)

# 取得想要关注的数据

## 基本数据切割

略。

## 反向选取行列

略。

## 条件选取数据

In [ ]:
import pandas as pd

pd.reset_option('all')
df = pd.read_csv('titanic_train.csv')

male_and_age_over_70 = (df.Sex == 'male') & (df.Age > 70)
(df[male_and_age_over_70]
    .style
    .applymap(lambda x: 'background-color: rgb(153, 255, 51)',
              subset=pd.IndexSlice[:, 'Sex':'Age']))

同样的，也可以使用`query`以实现同样的需求。

In [ ]:
age = 70
df.query("Age > @age & Sex == 'male'")

## 选择有空值的 column

In [ ]:
df[df.isnull().any(axis=1)].head()\
    .style.highlight_null()

备注：上面的`\`是为了不产生缩进错误。

## 选取或排除特定类型 column

①使用`select_dtypess`函数中的`include`参数

In [ ]:
df.select_dtypes(include='number').head()

②使用`exclude`参数

In [ ]:
df.select_dtypes(exclude='object')

## 针对特定值选取 column

如果想根据特定值取出样本，可以使用`isin`。

In [ ]:
tickets = ["SC/Paris 2123", "PC 17475"]
df[df.Ticket.isin(tickets)]

## 选取 Top-k 值得样本

步骤细解：

- 获取 Top-k 值所在的 index

- 根据所得 index ，使用 isin() 得到样本数据

获取 index 的方法有两种，一是`value_counts()`，一是`nlargest()`。

In [ ]:
top_k = 3
top_tickets = df.Ticket.value_counts()[:top_k]
# df.Ticket.value_counts().nlargest(top_k).index 同样可得到目标值的 index
df[df.Ticket.isin(top_tickets.index)].head()

## 找到符合特定字符串的样本

In [ ]:
df[df.Name.str.contains("Mr\.")].head()
# 此处使用的是正则表达式，因此要使用转义字符

## 使用正则表达式获取数据

In [ ]:
df_data = pd.util.testing.makeTimeDataFrame(freq='7D')
df_data.filter(regex="2000-02.*", axis=0)

## 获取从某时间点开始的区间样本

使用`first()`函数。

例如，若是想取出前三周的数据，可使用如下代码：

In [ ]:
# df_data.first("3W")
df_data.last("3W")

时间相关的简写字母资料如下：

| Code   | Description         | Code   | Description          |
|--------|---------------------|--------|----------------------|
| ``D``  | Calendar day        | ``B``  | Business day         |
| ``W``  | Weekly              |        |                      |
| ``M``  | Month end           | ``BM`` | Business month end   |
| ``Q``  | Quarter end         | ``BQ`` | Business quarter end |
| ``A``  | Year end            | ``BA`` | Business year end    |
| ``H``  | Hours               | ``BH`` | Business hours       |
| ``T``  | Minutes             |``BMS`` | Business month start |
| ``S``  | Seconds             |``BQS`` | Business quarter start|
| ``L``  | Milliseonds         |``BAS`` | Business year start  |
| ``U``  | Microseconds        | ``MS``  | Month start         |
| ``N``  | nanoseconds         | ``QS``  | Quarter start       |
| ``AS``  | Year start             


# 基本数据处理与转换

## 对某一 column 应用相同运算

略。

（`apply`函数搭配 Python 的匿名函数 lamda）

## 对每个样本做自定义运算

In [ ]:
df_titanic = df.copy()

d = {'male': "男性", 'female': '女性'}
def generate_desc(row):
    return f"一名 {row['Age']} 岁的{d[row['Sex']]}"

df["描述"] = df.apply(generate_desc, axis=1)
df.loc[:4, 'Sex':]

## 将连续值转换成分类数据

略。

## 将 DataFrame 随机切成两个子集

将 DataFrame 随机切成两个独立子集，其方法有 scikit-learn 的`train_test_split`或者 numpy 的`np.random.randn`，或者使用 Pandas 的`sample`函数。

In [ ]:
df_train = df_titanic.sample(frac=0.8, random_state=5566)
df_test = df_titanic.drop(df_train.index)

display(df_train.head())
display(df_test.head())

备注：使用这个解法的前提是该 DataFrame 的索引独一无二；注意设定`random_state`，以便于他人可重现你的结果。

## 用 SQL 的方式合并两个 DataFrames

略。

## 存取并操作每一个样本

使用`itertuples`函数以取出每个样本。

In [ ]:
df_city = pd.DataFrame({
    'state': ['密蘇里州', '亞利桑那州', '肯塔基州', '紐約州'],
    'city': ['堪薩斯城', '鳳凰城', '路易維爾', '紐約市']})

for row in df_city.itertuples(name='City'):
    print(f'{row.city}是{row.state}里面的一个城市')
    
from collections import namedtuple

City = namedtuple('City', ['Index', 'state', 'city'])
c = City(3, '紐約州', '紐約市')
c == row

# 简单汇总 & 分析数据

## 取出某 column 的 Top-k 的值

略。

## 一行描述数值 Column

In [ ]:
df.describe()

In [ ]:
df.describe().loc[['mean', 'std'], 'Survived':'Age']

## 找出 Column 中所有出现过的值

In [ ]:
df.Sex.unique()

## 分组汇总结果

In [ ]:
df.groupby("Pclass").Age.mean()

In [ ]:
df.groupby("Sex").Survived.describe()

In [ ]:
df.groupby(["Sex", "Pclass"]).size().unstack()

In [ ]:
df.groupby(['Sex', 'Pclass']).Age.agg(['min', 'max', 'count'])

In [ ]:
df.groupby(['Sex', 'Pclass']).Age.agg(['min', 'max', 'count']).unstack()

In [ ]:
df.pivot_table(index='Sex',
               columns='Pclass',
               values='Age',
               aggfunc=['min', 'max', 'count'])

## 结合原始数据与汇总结果

In [ ]:
df = df_titanic.copy()
df['Avg_age'] = df.groupby("Sex").Age.transform("mean")
df['Above_avg_age'] = df.apply(lambda x: 'yes' if x.Age > x.Avg_age else 'no', 
                               axis=1)
# styling
(df.loc[:4, 'Sex':]
 .style
 .highlight_max(subset=['Avg_age'])
 .applymap(lambda x: 'background-color: rgb(153, 255, 51)', 
           subset=pd.IndexSlice[[0, 4], ['Age', 'Above_avg_age']])
)

## 对时间数据做汇总

略。

## 简易绘图并修改预设样式

In [ ]:
import matplotlib.pyplot as plt

plt.style.available

# 其他工具推荐

## tqdm：可视化数据处理进度

### 安装方法

通过 Anaconda 安装：

In [ ]:
conda install -c conda-forge tqdm

### 使用示例

In [ ]:
from tqdm import tqdm_notebook
tqdm_notebook().pandas()


# 只需將 `apply` 替换成 `progress_apply`
df['存活'] = df.Survived.progress_apply(lambda x: '倖存' if x else '死亡')
df.loc[:5, 'Survived':'存活']

## swifter：加速处理数据

### 安装方法

通过 Anaconda 安装：

In [ ]:
conda install -c conda-forge swifter

### 使用示例

In [ ]:
import swifter
df = pd.DataFrame(pd.np.random.rand(1000000, 1), columns=['x'])

# 只需加上关键字 swifter
%timeit -n 10 df['x2'] = df['x'].apply(lambda x: x**2)
%timeit -n 10 df['x2'] = df['x'].swifter.apply(lambda x: x**2)

## qgrid：即时排序、筛选及编辑 DataFrame

### 安装方法

通过 Anaconda 安装：

In [ ]:
conda install -c tim_shawver qgrid
jupyter nbextension enable --py --sys-prefix widgetsnbextension

### 使用示例

In [ ]:
import qgrid
qgrid.set_grid_option('maxVisibleRows', 7)
q = qgrid.show_grid(df_titanic)
q

在线演示视频：https://leemeng.tw/images/pandas/qgrid_demo.mp4

## pandas-profiling：一键 EDA 神器

### 安装方法

通过 Anaconda 安装：

In [ ]:
conda install -c conda-forge pandas-profiling

### 使用示例

In [ ]:
import pandas_profiling
df = df_titanic.copy()

# 一行報表：將想觀察的 DataFrame 丟進去就完工了
pandas_profiling.ProfileReport(df)

In [ ]:
import pandas_profiling

In [ ]:
df = pd.read_csv("titanic_train.csv")
pandas_profiling.ProfileReport(df)